In [ ]:
import re
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from scipy.spatial import cKDTree

def load_vector_list(filename):
    coords, count = [], None
    pat = re.compile(r'\(\s*([-\d.eE]+)\s+([-\d.eE]+)\s+([-\d.eE]+)\s*\)')
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if count is None:
                m = re.match(r'^(\d+)$', line)
                if m:
                    count = int(m.group(1))
                continue
            if line in ('(', ')'):
                continue
            m = pat.match(line)
            if m:
                coords.append([float(m.group(1)), float(m.group(2)), float(m.group(3))])
            if count and len(coords) >= count:
                break
    return np.array(coords)

def load_scalar_list(filename):
    vals, count = [], None
    with open(filename, 'r') as f:
        in_block = False
        for line in f:
            s = line.strip()
            if count is None:
                m = re.match(r'^(\d+)$', s)
                if m:
                    count = int(m.group(1))
                continue
            if s == '(':
                in_block = True
                continue
            if s == ')':
                break
            if in_block:
                try:
                    vals.append(float(s))
                except ValueError:
                    pass
            if count and len(vals) >= count:
                break
    return np.array(vals)

# Load the 33,170 cell centres and nut field
coords_large = load_vector_list("C_33170.txt")
nut_large = load_scalar_list("nut.txt")

# Load the 1,354 target points
coords_small = load_vector_list("C_1354.txt")

# Detect and drop the flat dimension for 2D interpolation
spans = np.ptp(coords_large, axis=0)
drop = int(np.argmin(spans))
keep = [i for i in range(3) if i != drop]
pts2d_large = coords_large[:, keep]
pts2d_small = coords_small[:, keep]

# Perform linear interpolation
nut_small = griddata(pts2d_large, nut_large, pts2d_small, method="linear")

# Nearest-neighbour fallback
mask_nan = np.isnan(nut_small)
if mask_nan.any():
    tree = cKDTree(coords_large)
    _, idx = tree.query(coords_small[mask_nan], k=1)
    nut_small[mask_nan] = nut_large[idx]

# Scatter plot
plt.figure()
plt.scatter(pts2d_small[:,0], pts2d_small[:,1], c=nut_small)
plt.colorbar(label='nut')
plt.xlabel(f'Coordinate {keep[0]}')
plt.ylabel(f'Coordinate {keep[1]}')
plt.title('Interpolated nut Field on 1354 Points')
plt.show()
